### Connect to storage & load libraries

In [0]:
ServicePrincipalID = dbutils.secrets.get(scope="data-bricks", key="SPID")
ServicePrincipalKey = dbutils.secrets.get(scope="data-bricks", key="SPKey")
DirectoryID = dbutils.secrets.get(scope="data-bricks", key="DirectoryID")

Directory = f"https://login.microsoftonline.com/{DirectoryID}/oauth2/token"

container_name = 'my-csv'
storage_account_name = 'mainstorageaccountv2'

url=f'abfss://{container_name}@{storage_account_name}.dfs.core.windows.net'
mnt_path = '/mnt/test'

configs = {
  "fs.azure.account.auth.type": "OAuth",
  "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
  "fs.azure.account.oauth2.client.id": ServicePrincipalID,
  "fs.azure.account.oauth2.client.secret": ServicePrincipalKey,
  "fs.azure.account.oauth2.client.endpoint": Directory,
  "fs.azure.createRemoteFileSystemDuringInitialization": "false"
}

dbutils.fs.mount(
  source = url,
  mount_point = mnt_path,
  extra_configs = configs
)

Out[9]: True

In [0]:
# Import Libraries from pyspark
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.sql.functions import lit, col, date_format, to_date, add_months, coalesce,sum,substring,date_add,expr,datediff
from datetime import *

### List current directory (locally)

In [0]:
%sh
ls  /

bin
boot
BUILD
databricks
dbfs
dev
etc
home
lib
lib64
local_disk0
media
mnt
opt
proc
root
run
sbin
srv
sys
tmp
usr
var

### List mounted directory (storage in our case)

In [0]:
dbutils.fs.ls("/mnt/test")

Out[63]: [FileInfo(path='dbfs:/mnt/test/dataout/', name='dataout/', size=0),
 FileInfo(path='dbfs:/mnt/test/market_data.csv', name='market_data.csv', size=68870)]

In [0]:
dbutils.fs.ls("/") # In dbfs:/FileStore/ you will see many different test datasets

Out[12]: [FileInfo(path='dbfs:/FileStore/', name='FileStore/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/', name='databricks-datasets/', size=0),
 FileInfo(path='dbfs:/databricks-results/', name='databricks-results/', size=0),
 FileInfo(path='dbfs:/dataout/', name='dataout/', size=0),
 FileInfo(path='dbfs:/mnt/', name='mnt/', size=0),
 FileInfo(path='dbfs:/tmp/', name='tmp/', size=0),
 FileInfo(path='dbfs:/user/', name='user/', size=0)]

### Create dataframe and write it to storage

In [0]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('sparksession').getOrCreate()
data = [("James","Smith","USA","CA"),
    ("Michael","Rose","USA","NY"),
    ("Robert","Williams","USA","CA"),
    ("Maria","Jones","USA","FL")
  ]
columns = ["firstname","lastname","country","state"]
df = spark.createDataFrame(data = data, schema = columns)
df.show(truncate=True)

+---------+--------+-------+-----+
firstname|lastname|country|state|
+---------+--------+-------+-----+
 James| Smith| USA| CA|
 Michael| Rose| USA| NY|
 Robert|Williams| USA| CA|
 Maria| Jones| USA| FL|
+---------+--------+-------+-----+

In [0]:
table_file_path = '/mnt/test/foo.bar'
table_name = 'foo.bar'

In [0]:
(df
 .write
 .format('delta')
 .option('mergeSchema','true')
 .save(table_file_path)
)

In [0]:
df = spark.read.format('delta').load(table_file_path)
df.show(truncate=True)

+---------+--------+-------+-----+
firstname|lastname|country|state|
+---------+--------+-------+-----+
 Robert|Williams| USA| CA|
 Michael| Rose| USA| NY|
 James| Smith| USA| CA|
 Maria| Jones| USA| FL|
+---------+--------+-------+-----+

In [0]:
df_desc = spark.sql(f'DESCRIBE DETAIL "{table_file_path}"')
df_desc.show(truncate=False)

+------+------------------------------------+----+-----------+----------------------+-----------------------+-------------------+----------------+--------+-----------+----------+----------------+----------------+
format|id |name|description|location |createdAt |lastModified |partitionColumns|numFiles|sizeInBytes|properties|minReaderVersion|minWriterVersion|
+------+------------------------------------+----+-----------+----------------------+-----------------------+-------------------+----------------+--------+-----------+----------+----------------+----------------+
delta |6f710bc1-e6ee-40ee-8596-c29658c7b811|null|null |dbfs:/mnt/test/foo.bar|2021-04-26 16:27:17.172|2021-04-26 16:27:17|[] |4 |4568 |{} |1 |2 |
+------+------------------------------------+----+-----------+----------------------+-----------------------+-------------------+----------------+--------+-----------+----------+----------------+----------------+

In [0]:
df_hist = spark.sql(f'DESCRIBE HISTORY "{table_file_path}"')
df_hist.show(truncate=True)

+-------+-------------------+----------------+-----------------+---------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+
version| timestamp| userId| userName|operation| operationParameters| job| notebook| clusterId|readVersion| isolationLevel|isBlindAppend| operationMetrics|userMetadata|
+-------+-------------------+----------------+-----------------+---------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+
 0|2021-04-26 16:27:17|1215151417511068|gofr1@outlook.com| WRITE|{mode -> ErrorIfE...|null|{3312710932678982}|0329-064734-wikis718| null|WriteSerializable| true|{numFiles -> 4, n...| null|
+-------+-------------------+----------------+-----------------+---------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+

In [0]:
df_opt = spark.sql(f'OPTIMIZE "{table_file_path}"')
df_opt.show(truncate=False)

+-----------------+--------------------------------------------------------------------+
path |metrics |
+-----------------+--------------------------------------------------------------------+
/mnt/test/foo.bar|{0, 0, {null, null, 0.0, 0, 0}, {null, null, 0.0, 0, 0}, 0, null, 0}|
+-----------------+--------------------------------------------------------------------+

In [0]:
df_hist = spark.sql(f'DESCRIBE HISTORY "{table_file_path}"')
df_hist.show(truncate=True)

+-------+-------------------+----------------+-----------------+---------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+
version| timestamp| userId| userName|operation| operationParameters| job| notebook| clusterId|readVersion| isolationLevel|isBlindAppend| operationMetrics|userMetadata|
+-------+-------------------+----------------+-----------------+---------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+
 1|2021-04-26 16:27:47|1215151417511068|gofr1@outlook.com| OPTIMIZE|{predicate -> [],...|null|{3312710932678982}|0329-064734-wikis718| 0|SnapshotIsolation| false|{numRemovedFiles ...| null|
 0|2021-04-26 16:27:17|1215151417511068|gofr1@outlook.com| WRITE|{mode -> ErrorIfE...|null|{3312710932678982}|0329-064734-wikis718| null|WriteSerializable| true|{numFiles -> 4, n...| null|
+-------+-------------------+----------------+-----------------+---------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+

### Add new column to delta-table

In [0]:
df_new = df.withColumn('comment', lit(''))
df_new.write.format('delta').mode('append').option('mergeSchema','true').save(table_file_path)

In [0]:
df = spark.read.format('delta').load(table_file_path)
df.show(truncate=True)

+---------+--------+-------+-----+-------+
firstname|lastname|country|state|comment|
+---------+--------+-------+-----+-------+
 Robert|Williams| USA| CA| |
 Michael| Rose| USA| NY| |
 James| Smith| USA| CA| |
 Maria| Jones| USA| FL| |
 Robert|Williams| USA| CA| null|
 Michael| Rose| USA| NY| null|
 James| Smith| USA| CA| null|
 Maria| Jones| USA| FL| null|
+---------+--------+-------+-----+-------+

In [0]:
df_hist = spark.sql(f'DESCRIBE HISTORY "{table_file_path}"')
df_hist.show(truncate=True)

+-------+-------------------+----------------+-----------------+---------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+
version| timestamp| userId| userName|operation| operationParameters| job| notebook| clusterId|readVersion| isolationLevel|isBlindAppend| operationMetrics|userMetadata|
+-------+-------------------+----------------+-----------------+---------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+
 2|2021-04-26 16:28:16|1215151417511068|gofr1@outlook.com| WRITE|{mode -> Append, ...|null|{3312710932678982}|0329-064734-wikis718| 1|WriteSerializable| false|{numFiles -> 1, n...| null|
 1|2021-04-26 16:27:47|1215151417511068|gofr1@outlook.com| OPTIMIZE|{predicate -> [],...|null|{3312710932678982}|0329-064734-wikis718| 0|SnapshotIsolation| false|{numRemovedFiles ...| null|
 0|2021-04-26 16:27:17|1215151417511068|gofr1@outlook.com| WRITE|{mode -> ErrorIfE...|null|{3312710932678982}|0329-064734-wikis718| null|WriteSerializable| true|{numFiles -> 4, n...| null|
+-------+-------------------+----------------+-----------------+---------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+

### Get versions of the delta-table

In [0]:
df = (spark.read.format('delta').option('versionAsOf', 0).load(table_file_path))
df.show()

+---------+--------+-------+-----+
firstname|lastname|country|state|
+---------+--------+-------+-----+
 Robert|Williams| USA| CA|
 Michael| Rose| USA| NY|
 James| Smith| USA| CA|
 Maria| Jones| USA| FL|
+---------+--------+-------+-----+

In [0]:
df = (spark.read.format('delta').option('versionAsOf', 2).load(table_file_path))
df.show()

+---------+--------+-------+-----+-------+
firstname|lastname|country|state|comment|
+---------+--------+-------+-----+-------+
 Robert|Williams| USA| CA| |
 Michael| Rose| USA| NY| |
 James| Smith| USA| CA| |
 Maria| Jones| USA| FL| |
 Robert|Williams| USA| CA| null|
 Michael| Rose| USA| NY| null|
 James| Smith| USA| CA| null|
 Maria| Jones| USA| FL| null|
+---------+--------+-------+-----+-------+

In [0]:
dbutils.fs.ls("/mnt/test")

Out[83]: [FileInfo(path='dbfs:/mnt/test/dataout/', name='dataout/', size=0),
 FileInfo(path='dbfs:/mnt/test/foo.bar/', name='foo.bar/', size=0),
 FileInfo(path='dbfs:/mnt/test/market_data.csv', name='market_data.csv', size=68870)]

### Create database in Databricks

In [0]:
db_name = 'foo'

spark.sql(f"""CREATE DATABASE IF NOT EXISTS {db_name} COMMENT 'This is for test purposes' LOCATION '/user';""")

Out[84]: DataFrame[]

In [0]:
spark.sql(f'DESCRIBE DATABASE EXTENDED {db_name}').show(truncate=False)

+-------------------------+--------------------------+
database_description_item|database_description_value|
+-------------------------+--------------------------+
Database Name |foo |
Comment |This is for test purposes |
Location |dbfs:/user |
Owner |root |
Properties | |
+-------------------------+--------------------------+

### Create table in Databricks

In [0]:
spark.sql("""create table if not exists {} using delta location '{}'""".format(table_name, table_file_path))

Out[86]: DataFrame[]

In [0]:
df_check = spark.read.table(table_name)
df_check.show()

+---------+--------+-------+-----+-------+
firstname|lastname|country|state|comment|
+---------+--------+-------+-----+-------+
 Robert|Williams| USA| CA| |
 Michael| Rose| USA| NY| |
 James| Smith| USA| CA| |
 Maria| Jones| USA| FL| |
 Robert|Williams| USA| CA| null|
 Michael| Rose| USA| NY| null|
 James| Smith| USA| CA| null|
 Maria| Jones| USA| FL| null|
+---------+--------+-------+-----+-------+

### Drop table in Databricks

In [0]:
spark.sql("""drop table if exists {}""".format(table_name))

Out[91]: DataFrame[]

### Remove table from storage

In [0]:
dbutils.fs.rm(table_file_path, recurse=True)

Out[92]: True

### Read csv from storage

In [0]:
file_path = '/mnt/test/market_data.csv'
db_table = 'foo.market_data'

In [0]:
df_csv = (spark.read.format("csv")
       .option('header','true')
       .option('delimiter',',')
       .option('inferSchema', 'true')
       #.option('format', '%i')
       .load(file_path)
      )

df_csv.show()

+---------+-----+-----+-----+----------+----------+-----+----------+---------+
 Date| EPAM| XOM| VIX| AAPL| FB| AMJ| GOOG| ICHGF|
+---------+-----+-----+-----+----------+----------+-----+----------+---------+
16-Feb-16|60.44|81.22|24.11| 96.639999|101.610001|23.44|717.640015| 33.0|
12-Feb-16| 57.0|81.03| 25.4| 93.989998|102.010002|22.05|706.890015| 33.0|
11-Feb-16|58.05| 79.6|28.14| 93.699997|101.910004|20.88|706.359985| 33.0|
10-Feb-16|58.06|79.35|26.29| 94.269997| 101.0|21.84|706.849976| 33.0|
 9-Feb-16|58.69|80.08|26.54| 94.989998| 99.540001|21.68| 701.02002|36.790001|
 8-Feb-16|59.77|81.16| 26.0| 95.010002| 99.75|22.55|704.159973|36.790001|
 5-Feb-16|60.92|80.08|23.38| 94.019997| 104.07|24.53| 703.76001|36.790001|
 4-Feb-16|68.86|79.83|21.84| 96.599998|110.489998|25.24|730.030029|36.790001|
 3-Feb-16|71.71|78.48|21.65| 96.349998|112.690002| 24.8|749.380005|36.790001|
 2-Feb-16| 75.2|74.59|21.98| 94.480003|114.610001|24.74|780.909973|36.790001|
 1-Feb-16|75.86|76.29|19.98| 96.43|115.089996|25.24| 770.77002|36.790001|
29-Jan-16| 74.9|77.85| 20.2| 97.339996|112.209999|25.66|761.349976|36.790001|
28-Jan-16| 71.5|76.99|22.42| 94.089996|109.110001|25.39|748.299988|36.790001|
27-Jan-16|70.67|75.29|23.11| 93.419998| 94.449997|24.31|717.580017|36.790001|
26-Jan-16|70.99| 76.7| 22.5| 99.989998| 97.339996|24.44|733.789978|36.790001|
25-Jan-16|72.49|73.98|24.15| 99.440002| 97.010002|23.52|733.619995|36.790001|
22-Jan-16|72.12|76.57|22.34|101.419998| 97.940002|24.22|745.460022|36.790001|
21-Jan-16|70.18| 74.1|26.69| 96.300003| 94.160004|22.36|726.669983|36.790001|
20-Jan-16|68.77|73.18|27.59| 96.790001| 94.349998|21.32|718.559998|36.790001|
19-Jan-16|67.84| 76.4|26.05| 96.660004| 95.260002|22.67|719.080017|36.790001|
+---------+-----+-----+-----+----------+----------+-----+----------+---------+
only showing top 20 rows

### End of the notebook